
# <center><font color= #F08080 > <b>CU53_impacto de las políticas de inversión en sanidad, infraestructuras y promoción turística en el SPI</font></center>

#  <font color='green'>IV. Model development</font>

En este anexo se incluye el código utilizado durante el desarrollo 
de los modelos incluidos en el caso de uso.

## Modelo RIDGE



In [1]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

### Paquetes

In [2]:

## Modelos CU 53
library(glmnet)
library(dplyr)
library(readr)

Loading required package: Matrix

Loaded glmnet 4.1-6


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Datos

In [3]:
inversiones <- read_csv("CU_53_05_05_inversiones_cm.csv")

spi <- read_csv("CU_53_05_02_01_spi.csv")  
spi_meta <- read_csv("CU_53_05_02_02_spi_metadata.csv")

escenario_spi <- read_csv("ESCENARIO_INVERSIONES_PAISES.csv")

Rows: 24 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): grupo
dbl (2): anyo, inversion

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2364 Columns: 81
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): country, spicountrycode, status
dbl (78): rank_score_spi, spiyear, score_spi, score_bhn, score_fow, score_op...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 81 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): id_var, name_var, role, Dimension, Component, Unit of measurement, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column typ

## Modelo

In [4]:
dfmodel <- spi |> 
  filter(spicountrycode != "WWW") |> 
  filter(!is.na(score_spi)) |> 
  select(spicountrycode, spiyear, score_spi) |> 
    inner_join(escenario_spi)
x <- dfmodel |> select(-c(spicountrycode, score_spi) ) |> as.matrix()
y <- dfmodel |> select(score_spi)  |> as.matrix()

## Modelo
cv_model <- cv.glmnet(x, y, alpha = 0)

#find optimal lambda value that minimizes test MSE
best_lambda <- cv_model$lambda.min

## MOSTRAR EN INTERFAZ
best_lambda

# plot(cv_model) 

best_model <- glmnet(x, y, alpha = 0, lambda = best_lambda)
cc <- coef(best_model)


## MOSTRAR EN INTERFAZ
cc



Joining with `by = join_by(spicountrycode, spiyear)`


[1] 1.480453

5 x 1 sparse Matrix of class "dgCMatrix"
                      s0
(Intercept) 29.708657618
spiyear     -0.003411833
inv_inf      0.801822794
inv_tur     14.972654971
inv_san     19.117756691

## Predicción

In [5]:
## Predicción escenario región

escenario_region <- read_csv("ESCENARIO_INVERSIONES_REGION.csv")

prediccion <- predict(best_model, escenario_region |> makeX())

## Representar series: escenario + predicción

## guardar modelo

write_rds(best_model, "modelo_reg.rds")


Rows: 4 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): spiyear, inv_inf, inv_san, inv_tur

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
